In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data=pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_data=pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
print(train_data.shape)
print(test_data.shape)

In [4]:
train_data

In [7]:
train_Y=train_data['SalePrice']
train_X=train_data.drop('SalePrice',axis=1)

In [11]:
train_X.shape,train_Y.shape

In [13]:
#setting all non-numeric data types to categorical
for col,content in train_X.items():
    if not pd.api.types.is_numeric_dtype(content):
        train_X[col]=train_X[col].astype('category')

In [36]:
#test_data
for col,content in test_data.items():
    if not pd.api.types.is_numeric_dtype(content):
        test_data[col]=test_data[col].astype('category')

In [16]:
train_X.info()

In [23]:
for col, content in train_X.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isna(content).sum():
            train_X[col].fillna(train_X[col].median(), inplace=True)

In [37]:
#test_data
for col, content in test_data.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isna(content).sum():
            test_data[col].fillna(test_data[col].median(), inplace=True)

In [31]:
for col, content in train_X.items():
    if not pd.api.types.is_numeric_dtype(content):
        train_X[col]=pd.Categorical(content).codes+1

In [38]:
#test_data
for col, content in test_data.items():
    if not pd.api.types.is_numeric_dtype(content):
        test_data[col]=pd.Categorical(content).codes+1

In [32]:
train_X.info()

In [33]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs=-1)


from sklearn.model_selection import RandomizedSearchCV
grid = {'n_estimators':[100,125,150,200,500],
       'max_depth' : [None,2,5,8,10,15],
       }

model_rs = RandomizedSearchCV(estimator=model, n_iter=10, param_distributions=grid,
                             cv=5, verbose=2)
model_rs.fit(train_X, train_Y)

In [34]:
model_rs.best_params_

In [35]:
model_rs.score(train_X,train_Y)

In [39]:
predictions=model_rs.predict(test_data)

In [40]:
predictions[:10]

In [41]:
submission =pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
final_submission = pd.DataFrame({'Id':submission['Id'],'SalePrice':predictions})
final_submission.to_csv('submission.csv', index=False)